<a href="https://colab.research.google.com/github/zahradm/Thesis/blob/main/AdversarialTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers textattack sentence_transformers torchfile evaluate 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import transformers
import textattack
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoModel
from transformers import pipeline, AutoModelForTokenClassification, AutoModelForSequenceClassification
from transformers import AutoTokenizer
from textattack.augmentation.recipes import  *
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Creating adversarial data


In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-rotten-tomatoes")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/roberta-base-rotten-tomatoes", truncation=True)
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--textattack--roberta-base-rotten-tomatoes/snapshots/3fd77299e29adc4fe8c5f181810a6a1e7498dbd4/config.json
Model config RobertaConfig {
  "_name_or_path": "textattack/roberta-base-rotten-tomatoes",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "finetuning_task": "rotten_tomatoes",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file pytorch_model.bin from cache at 

In [ ]:
dataset = load_dataset("rotten_tomatoes", split="train")
sorted_dataset = dataset.sort('label')
shuffled_dataset = sorted_dataset.shuffle(seed=45)
shuffled_dataset = textattack.datasets.HuggingFaceDataset(shuffled_dataset)

In [ ]:
attack = textattack.attack_recipes.BAEGarg2019.build(model_wrapper)
# Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=3000, log_to_csv="/content/drive/MyDrive/Thesis/log_BAEGarg2019_3.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, shuffled_dataset, attack_args)
attacker.attack_dataset()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pytorch_model.bin
Generate config GenerationConfig {
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of BertForMaskedLM were initialized from the model checkpoint at bert-base-uncas

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_ac

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.936338023
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 



...

[Succeeded / Failed / Skipped / Total] 1778 / 1123 / 89 / 2990: 100%|█████████▉| 2990/3000 [2:05:18<00:25,  2.51s/it]textattack: Saving checkpoint under "checkpoints/1676232671905.ta.chkpt" at 2023-02-12 20:11:11 after 2990 attacks.


[Succeeded / Failed / Skipped / Total] 1782 / 1124 / 89 / 2995: 100%|█████████▉| 2995/3000 [2:05:31<00:12,  2.51s/it]textattack: Saving checkpoint under "checkpoints/1676232685119.ta.chkpt" at 2023-02-12 20:11:25 after 2995 attacks.


[Succeeded / Failed / Skipped / Total] 1785 / 1126 / 89 / 3000: 100%|██████████| 3000/3000 [2:05:46<00:00,  2.52s/it]textattack: Saving checkpoint under "checkpoints/1676232700012.ta.chkpt" at 2023-02-12 20:11:40 after 3000 attacks.
[Succeeded / Failed / Skipped / Total] 1785 / 1126 / 89 / 3000: 100%|██████████| 3000/3000 [2:05:46<00:00,  2.52s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1785   |
| Number of failed attacks:     | 1126   |
| Number of skipped attacks:    | 89     |
| Original accuracy:            | 97.03% |
| Accuracy under attack:        | 37.53% |
| Attack success rate:          | 61.32% |
| Average perturbed word %:     | 14.24% |
| Average num. words per input: | 18.49  |
| Avg num queries:              | 53.48  |
+-------------------------------+--------+


 ...]

# Adversarial data preprocess

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Thesis/log_BAEGarg2019_3.csv')

def success_sample(data):
  selected_example = data.loc[data['result_type']=='Successful']
  selected_example.reset_index(inplace=True)
  return selected_example

def clear_data(data):
  data['original_text'] = data['original_text'].astype('str').str.replace("[", "", regex=True).astype('str')
  data['original_text'] = data['original_text'].astype('str').str.replace("]", "", regex=True).astype('str')
  data['perturbed_text'] = data['perturbed_text'].astype('str').str.replace("[", "", regex=True).astype('str')
  data['perturbed_text'] = data['perturbed_text'].astype('str').str.replace("]", "", regex=True).astype('str')
  return data

In [ ]:
succsess_adv = success_sample(data)
succsess_adv

,index,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,0,"in between the icy stunts , the actors [[spout...","in between the icy stunts , the actors [[provi...",0.097313,0.672674,0,1,0,57,Successful
1,1,a [[spiffy]] animated feature about an unruly ...,a [[brief]] animated feature about an unruly a...,0.036418,0.523241,1,0,1,47,Successful
2,2,duvall is [[strong]] as always .,duvall is [[silent]] as always .,0.005091,0.990095,1,0,1,38,Successful
3,3,more [[maudlin]] than [[sharp]] .,more [[tender]] than [[usual]] .,0.004862,0.973415,0,1,0,33,Successful
4,5,"a timid , [[soggy]] near [[miss]] .","a timid , [[quite]] near [[friend]] .",0.007871,0.864187,0,1,0,52,Successful
...,...,...,...,...,...,...,...,...,...,...
1780,2993,"for those of an indulgent , slightly sunbaked ...","for those of an indulgent , slightly sunbaked ...",0.193271,0.501063,1,0,1,48,Successful
1781,2994,"the [[film]] has [its] [[moments]] , but they ...","the [[journey]] has [its] [[limits]] , but the...",0.100865,0.825219,0,1,0,100,Successful
1782,2996,[it's] a [[clever]] thriller with enough unexp...,[it's] a [[vampire]] thriller with enough unex...,0.008037,0.791926,1,0,1,71,Successful
1783,2998,it's one of those baseball pictures where the ...,it's one of those baseball pictures where the ...,0.052917,0.518758,0,1,0,41,Successful


In [ ]:
adv_data = clear_data(succsess_adv)
adv_data

<ipython-input-5-84d7f6f5ac76>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['original_text'] = data['original_text'].astype('str').str.replace("[", "", regex=True).astype('str')
<ipython-input-5-84d7f6f5ac76>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['original_text'] = data['original_text'].astype('str').str.replace("]", "", regex=True).astype('str')
<ipython-input-5-84d7f6f5ac76>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

,index,original_text,perturbed_text,original_score,perturbed_score,original_output,perturbed_output,ground_truth_output,num_queries,result_type
0,0,"in between the icy stunts , the actors spout h...","in between the icy stunts , the actors provide...",0.097313,0.672674,0,1,0,57,Successful
1,1,a spiffy animated feature about an unruly adol...,a brief animated feature about an unruly adole...,0.036418,0.523241,1,0,1,47,Successful
2,2,duvall is strong as always .,duvall is silent as always .,0.005091,0.990095,1,0,1,38,Successful
3,3,more maudlin than sharp .,more tender than usual .,0.004862,0.973415,0,1,0,33,Successful
4,5,"a timid , soggy near miss .","a timid , quite near friend .",0.007871,0.864187,0,1,0,52,Successful
...,...,...,...,...,...,...,...,...,...,...
1780,2993,"for those of an indulgent , slightly sunbaked ...","for those of an indulgent , slightly sunbaked ...",0.193271,0.501063,1,0,1,48,Successful
1781,2994,"the film has its moments , but they are few an...","the journey has its limits , but they are few ...",0.100865,0.825219,0,1,0,100,Successful
1782,2996,it's a clever thriller with enough unexpected ...,it's a vampire thriller with enough unexpected...,0.008037,0.791926,1,0,1,71,Successful
1783,2998,it's one of those baseball pictures where the ...,it's one of those baseball pictures where the ...,0.052917,0.518758,0,1,0,41,Successful


In [ ]:
adv_data = adv_data[['perturbed_text', 'ground_truth_output']]
adv_data

,perturbed_text,ground_truth_output
0,"in between the icy stunts , the actors provide...",0
1,a brief animated feature about an unruly adole...,1
2,duvall is silent as always .,1
3,more tender than usual .,0
4,"a timid , quite near friend .",0
...,...,...
1780,"for those of an indulgent , slightly sunbaked ...",1
1781,"the journey has its limits , but they are few ...",0
1782,it's a vampire thriller with enough unexpected...,1
1783,it's one of those baseball pictures where the ...,0


In [ ]:
adv_data.columns = ['text', 'label']
adv_data

,text,label
0,"in between the icy stunts , the actors provide...",0
1,a brief animated feature about an unruly adole...,1
2,duvall is silent as always .,1
3,more tender than usual .,0
4,"a timid , quite near friend .",0
...,...,...
1780,"for those of an indulgent , slightly sunbaked ...",1
1781,"the journey has its limits , but they are few ...",0
1782,it's a vampire thriller with enough unexpected...,1
1783,it's one of those baseball pictures where the ...,0


# Adcersarial training and saving model

In [ ]:
#path = '/content/drive/MyDrive/Thesis/log_BAEGarg2019_3.csv'
#adv_data = pd.read_csv(path)
#data = data.drop(['Unnamed: 0','index'], axis=1)
df = pd.DataFrame(adv_data)
rotten_tomatoes = load_dataset("rotten_tomatoes")

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Dataset rotten_tomatoes downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
df

,text,label
0,"in between the icy stunts , the actors provide...",0
1,a brief animated feature about an unruly adole...,1
2,duvall is silent as always .,1
3,more tender than usual .,0
4,"a timid , quite near friend .",0
...,...,...
1780,"for those of an indulgent , slightly sunbaked ...",1
1781,"the journey has its limits , but they are few ...",0
1782,it's a vampire thriller with enough unexpected...,1
1783,it's one of those baseball pictures where the ...,0


In [ ]:
adv_split_index = round(len(df)/10)*8
adv_train = df[:adv_split_index]
adv_test = df[adv_split_index:]

In [ ]:
def shuffle(dataName, split):
  dataset = load_dataset(dataName, split=split)
  sorted_dataset = dataset.sort('label')
  shuffled_dataset = sorted_dataset.shuffle(seed=45)
  return shuffled_dataset


In [ ]:
shuffle_train = shuffle('rotten_tomatoes','train')
shuffle_test = shuffle('rotten_tomatoes', 'validation')
benign_train = shuffle_train[:round(len(shuffle_train)/5)]
benign_test = shuffle_test[:round(len(shuffle_test)/5)]

In [ ]:
benign_train_df = pd.DataFrame(list(zip(benign_train['text'], benign_train['label'])), columns =['text', 'label']) 
benign_test_df = pd.DataFrame(list(zip(benign_test['text'], benign_test['label'])), columns =['text', 'label']) 

In [ ]:
df_merged_train = pd.concat([benign_train_df, adv_train], ignore_index=True, sort=False)
df_merged_test = pd.concat([benign_test_df, adv_test], ignore_index=True, sort=False)

In [ ]:
all_train = Dataset.from_pandas(df_merged_train)
all_test = Dataset.from_pandas(df_merged_test)

In [ ]:
model_name = "textattack/roberta-base-rotten-tomatoes"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets_train = all_train.map(tokenize_function, batched=True)
tokenized_datasets_test = all_test.map(tokenize_function, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(output_dir="/content/drive/MyDrive/Thesis_roberta_BAG", evaluation_strategy="epoch", num_train_epochs=8)

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model("/content/drive/MyDrive/Thesis_roberta_BAG")

Some weights of the model checkpoint at textattack/roberta-base-rotten-tomatoes were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3130
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3136
  Number of trainable parameters = 124647170


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.800515,0.635889
2,0.575200,1.177879,0.613240
3,0.378900,1.867669,0.621951
4,0.211400,2.128972,0.663763
5,0.211400,2.221284,0.693380
6,0.073500,2.309239,0.721254
7,0.022200,2.869697,0.675958
8,0.006500,2.833451,0.679443


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 574
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/Thesis_roberta_BAG/checkpoint-500
Configuration saved in /content/drive/MyDrive/Thesis_roberta_BAG/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/Thesis_roberta_BAG/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 574
  Batch size = 8
Saving model checkpoint to /content/drive/MyDrive/Thesis_rob

In [6]:
# load the model
from transformers import AutoModelForSequenceClassification
load_model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Thesis_roberta_BAG")
tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-rotten-tomatoes")


# Adversarial attack on trained model

In [7]:
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(load_model, tokenizer)
attack = textattack.attack_recipes.BAEGarg2019.build(model_wrapper)
dataset_test = textattack.datasets.HuggingFaceDataset("rotten_tomatoes", split="test")
attack_args = textattack.AttackArgs(num_examples=1066, checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset_test, attack_args)
attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

...

[Succeeded / Failed / Skipped / Total] 514 / 347 / 199 / 1060:  99%|█████████▉| 1060/1066 [37:41<00:12,  2.13s/it]textattack: Saving checkpoint under "checkpoints/1677345807704.ta.chkpt" at 2023-02-25 17:23:27 after 1060 attacks.


[Succeeded / Failed / Skipped / Total] 516 / 348 / 201 / 1065: 100%|█████████▉| 1065/1066 [37:48<00:02,  2.13s/it]textattack: Saving checkpoint under "checkpoints/1677345814596.ta.chkpt" at 2023-02-25 17:23:34 after 1065 attacks.


[Succeeded / Failed / Skipped / Total] 517 / 348 / 201 / 1066: 100%|██████████| 1066/1066 [37:51<00:00,  2.13s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 517    |
| Number of failed attacks:     | 348    |
| Number of skipped attacks:    | 201    |
| Original accuracy:            | 81.14% |
| Accuracy under attack:        | 32.65% |
| Attack success rate:          | 59.77% |
| Average perturbed word %:     | 14.98% |
| Average num. words per input: | 18.65  |
| Avg num queries:              | 55.17  |
+-------------------------------+--------+


 ...]